In [181]:
import json
import re
import pandas as pd
import nltk
from nltk.stem.lancaster import LancasterStemmer
from autocorrect import spell 
from collections import Counter

import time

st = LancasterStemmer()

In [144]:
ori_review_path= r"D:\Courses\Big Data Tools and Methods CS-696\review.json"
review_path= r"./smaller_review.json"
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [53]:
def findwordsandcount(df, review_count):
#     count_total = 0
#     date, row in df.T.iteritems()
    for index, row in df.T.iteritems():
        new_string = re.sub(r"[^a-zA-Z\ ]+"," ", row["text"])
        tokens = nltk.word_tokenize(new_string)
        
        df.at[index,'count'] = len(tokens) 
        if len(tokens) >=review_count:
            df.at[index,'text'] = removeletter(tokens)

In [146]:
def removeletter(tokens):
    save_token =[]
    for atoken in tokens:
        if (len(atoken) != 1) | (atoken == 'a') | (atoken == 'i') \
        | (atoken == 'A')| (atoken == 'I'):
            atoken = spell(atoken)
            atoken = str.lower(atoken)
            atoken = st.stem(atoken)
            save_token.append(atoken)
#         if (atoken == 'a') | (atoken == 'i'):
#             save_token.append(atoken)
    return save_token

In [88]:
def finduniqueword(df):
    all_reiew = []
#     unique_words = []
    for index, row in df.T.iteritems():
        all_reiew  = all_reiew + df.at[index,'text']
        all_reiew = list(set(all_reiew))
#         all_reiew = set(all_reiew)
#         my_set  =set(all_reiew)
#         print(all_reiew)
#         unique_words = set(all_reiew)
    return all_reiew

In [193]:
def countwords(df,unique_word):
    dataset_df = pd.DataFrame(columns=['review_id','stars'] +unique_word )

    for index, a_review in df.iterrows():
        empty_row = dict(zip(['review_id','stars'] +unique_word , [0]*(len(unique_word)+2)) )
        counts  = Counter(a_review.text)

        empty_row['review_id'] = a_review.review_id
        empty_row['stars'] = a_review.stars
        empty_row.update(counts)
        dataset_df.loc[index] = empty_row
    return dataset_df

In [205]:
count = 0

words_list = []
cols = ['review_id','stars','text']

all_dataset_df = pd.DataFrame(columns=['review_id','stars'])
start_time = time.time()
for part_df in pd.read_json(review_path, lines=True, chunksize=100):
    part_df = part_df[cols]
    part_df['count']  =0
    findwordsandcount(part_df,80)
    print("findwordsandcount: ", (time.time() - start_time))
    start_time = time.time()
    
    part_df = part_df.loc[part_df['count'] >= 80]
    unique_word = finduniqueword(part_df)
    unique_word.sort()
    
    print("finduniqueword: ", (time.time() - start_time))
    start_time = time.time()

    dataset_df = countwords(part_df,unique_word)
    
    all_dataset_df = all_dataset_df.append(dataset_df,sort=False)
    print("countwords: ", (time.time() - start_time))
    start_time = time.time()


findwordsandcount:  6.205396890640259
finduniqueword:  0.008978605270385742
countwords:  8.122785329818726
findwordsandcount:  7.155855894088745
finduniqueword:  0.007978677749633789
countwords:  7.634573221206665
findwordsandcount:  4.570805788040161
finduniqueword:  0.0059833526611328125
countwords:  6.419795751571655
findwordsandcount:  3.7878947257995605
finduniqueword:  0.007416963577270508
countwords:  6.427836656570435
findwordsandcount:  5.280874490737915
finduniqueword:  0.008974313735961914
countwords:  7.628591537475586
findwordsandcount:  1.4651143550872803
finduniqueword:  0.005980014801025391


KeyboardInterrupt: 

In [208]:
# all_dataset_df = 
# all_dataset_df = pd.DataFrame({'b':[5]})
# all_dataset_df = all_dataset_df.append(pd.DataFrame({'a':[5]}), ignore_index= True,sort=True)
all_dataset_df = all_dataset_df.fillna(0)
all_dataset_df

,review_id,stars,a,abl,about,absolv,ac,accommod,accomod,accord,...,waterfront,waterwork,wherev,whirl,witht,wyn,xo,yard,yolk,zinco
1,GJXCdrto3ASJOqKeVWPi6Q,5,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2TzJjDVDEuAW6MR5Vuc1ug,5,2,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11a8sVPMUFtaC7_ABRkmtw,1,11,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,fdiNeiN_hoCxCMy2wTRW9g,4,7,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,G7XHMxG0bx9oBJNECG4IFg,3,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,qrffudO73zsslZbe8B9D3Q,2,6,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,kbtscdyz6lvrtGjD1quQTg,4,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,-I5umRTkhw15RqpKMl_o1Q,1,4,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,Z7wgXp98wYB57QdRY3HQ3w,4,8,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,qlXw1JQ0UodW7qrmVgwCXw,4,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# ========================================================

In [148]:
partial_df['count']  =0
findwordsandcount(partial_df,80)
partial_df = partial_df.loc[partial_df['count'] >= 80]
partial_df
# len(partial_df.loc[partial_df['count'] >= 80])
# partial_df

,review_id,stars,text,count
1,GJXCdrto3ASJOqKeVWPi6Q,5,"[i, ad, trav, at, the, hard, rock, new, kel, c...",282
2,2TzJjDVDEuAW6MR5Vuc1ug,5,"[i, hav, to, say, that, thi, off, real, has, i...",108
4,11a8sVPMUFtaC7_ABRkmtw,1,"[today, was, my, second, out, of, three, sess,...",671
5,fdiNeiN_hoCxCMy2wTRW9g,4,"[i, ll, be, the, first, to, admit, that, i, wa...",301
6,G7XHMxG0bx9oBJNECG4IFg,3,"[tracy, dessert, had, a, big, nam, in, hong, k...",241
8,qrffudO73zsslZbe8B9D3Q,2,"[i, was, real, look, forward, to, visit, aft, ...",263
10,kbtscdyz6lvrtGjD1quQTg,4,"[lik, walk, back, in, tim, every, saturday, mo...",131
11,-I5umRTkhw15RqpKMl_o1Q,1,"[walk, in, around, on, a, friday, afternoon, w...",85
12,Z7wgXp98wYB57QdRY3HQ3w,4,"[wow, so, surpr, at, the, on, and, two, star, ...",209
13,qlXw1JQ0UodW7qrmVgwCXw,4,"[michael, from, red, carpet, vip, is, amaz, i,...",85


In [151]:
unique_word = finduniqueword(partial_df)
unique_word.sort()
# len(unique_word)
# unique_word

In [175]:
dataset_df = pd.DataFrame(columns=['review_id','stars'] +unique_word )

for index, a_review in partial_df.iterrows():
    empty_row = dict(zip(['review_id','stars'] +unique_word , [0]*(len(unique_word)+2)) )
    counts  = Counter(a_review.text)
    
    empty_row['review_id'] = a_review.review_id
    empty_row['stars'] = a_review.stars
    empty_row.update(counts)
    dataset_df.loc[index] = empty_row
dataset_df


,review_id,stars,a,ab,aback,abandon,abbrevy,abl,abord,about,...,zero,zest,zinco,zing,zippy,zit,zomby,zon,zucchin,zy
1,GJXCdrto3ASJOqKeVWPi6Q,5,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2TzJjDVDEuAW6MR5Vuc1ug,5,2,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,11a8sVPMUFtaC7_ABRkmtw,1,11,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,fdiNeiN_hoCxCMy2wTRW9g,4,7,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,1,0
6,G7XHMxG0bx9oBJNECG4IFg,3,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,qrffudO73zsslZbe8B9D3Q,2,6,0,0,0,0,2,0,1,...,0,0,0,0,0,0,0,0,0,0
10,kbtscdyz6lvrtGjD1quQTg,4,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,-I5umRTkhw15RqpKMl_o1Q,1,4,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
12,Z7wgXp98wYB57QdRY3HQ3w,4,8,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
13,qlXw1JQ0UodW7qrmVgwCXw,4,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [177]:
dataset_df.to_csv('./data.csv')